In [1]:
#윈도우 자체의 모듈과 그 밖의 모듈
import sys
import time
import os
import lxml
import re
import pandas as pd
import numpy as np
import urllib
import cv2
import pytesseract
import glob
from PIL import Image, ImageCms
from win32process import CREATE_NO_WINDOW

In [415]:
def gausThresh():
    # Lab의 L값을 저장할 리스트
    labVal = []
    # 사진 경로 확정
    path_dir = './input'
    file_list = os.listdir(path_dir)
    i = 0
    images = glob.glob('input\*.jpg')
    
    # Lab의 L값을 찾아 리스트에 저장
    for fname in images:
        coloredImg = cv2.imread(fname, 1)
        gausImage = cv2.cvtColor(coloredImg, cv2.COLOR_BGR2LAB)
        l_channel,a_channel,b_channel = cv2.split(gausImage)
        labVal.append(np.max(l_channel)) 
        
    # L값을 임계치로 지정하여 국소 영역 45*45 내 가중치를 적용하여 영상 이진화 진행 후 저장
    for gname in images:
        fname = cv2.imread(gname, 0)
        gausImg =cv2.adaptiveThreshold(fname,labVal[i],cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,45,2)
        cv2.imwrite('./output\\1.gausThresh\\' + str(file_list[i]), gausImg)
        i += 1
    
    # edgeDet에 L값이 필요하기 때문에 이를 반환
    return labVal

In [416]:
def edgeDet(labVal):
    labVal = []
    path_dir = './input'
    file_list = os.listdir(path_dir)
    i = 0
    images = glob.glob('input\*.jpg')

    for fname in images:
        coloredImg = cv2.imread(fname, 1)
        gausImage = cv2.cvtColor(coloredImg, cv2.COLOR_BGR2LAB)
        l_channel,a_channel,b_channel = cv2.split(gausImage)
        labVal.append(np.max(l_channel)) 
        
    for fname in images:
        
        im = cv2.imread(fname)
        imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        ret, thresh = cv2.threshold(imgray, 127, int(labVal[i]), 0)
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        cv2.drawContours(im, contours, -1, (0,int(labVal[i]),0), 3)
        cv2.imwrite('./output\\2.edgeDet\\' + str(file_list[i]), im)
        i += 1

In [417]:
def clahe():
    path_dir = './input'
    file_list = os.listdir(path_dir)
    i = 0
    images = glob.glob('input\*.jpg')
    for fname in images:
        coloredImg = cv2.imread(fname,1)
        coloredImg2 = cv2.imread(fname,0)
        
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        cla = clahe.apply(coloredImg2)
        clac = cv2.split(cla)
        clacTotal = int(np.average(clac))
        #print ('clacTotal: ' + str(clacTotal))

        hsvc = cv2.split(coloredImg)
        hsvTotal = int(np.average(hsvc))
        #print ('hsvTotal: ' + str(hsvTotal))
        
        total = clacTotal - hsvTotal
        
        lower_black = np.array([total,total,total])
        upper_black = np.array([clacTotal,clacTotal,clacTotal])
        mask = cv2.inRange(coloredImg, lower_black, upper_black)
        res = cv2.bitwise_and(coloredImg,coloredImg, mask= mask)
        
        median = cv2.blur(res,(11,11))
        ret,medianb = cv2.threshold(median, hsvTotal+1, 255, cv2.THRESH_BINARY)
        
        grayImage = cv2.cvtColor(medianb, cv2.COLOR_BGR2GRAY)
        ret, fImage = cv2.threshold(grayImage, hsvTotal+1, 255, cv2.THRESH_BINARY)
        
        cv2.imwrite('./output\\3.clahe\\' + str(file_list[i]), fImage)
        i += 1

In [418]:
def findCrack():
    gausThresh()
    edgeDet(labVal)
    clahe()
    
    i = glob.glob('input\*.jpg')
    k = 0
    for fanme in i:
        path_dir = './input'
        file_list = os.listdir(path_dir)
        Images = glob.glob('./input\\*.jpg')
        Image = cv2.imread(Images[k],1)
        
        green = cv2.imread('green.jpg', cv2.IMREAD_COLOR)
        
        gaus_path_dir = './output/1.gausThresh'
        gaus_file_list = os.listdir(path_dir)
        gausImages = glob.glob('./output\\1.gausThresh\\*.jpg')
        gausImage = cv2.imread(gausImages[k],1)
        
        edge_path_dir = './output/2.edgeDet'
        edge_file_list = os.listdir(path_dir)
        edgeImages = glob.glob('./output\\2.edgeDet\\*.jpg')
        edgeImage = cv2.imread(edgeImages[k],1)
        
        clahe_path_dir = './output/3.clahe'
        clahe_file_list = os.listdir(path_dir)
        claheImages = glob.glob('./output\\3.clahe\\*.jpg')
        claheImage = cv2.imread(claheImages[k],1)
        
        res = cv2.bitwise_and(edgeImage,claheImage);
        res2 = cv2.bitwise_and(gausImage,res);
        res3 = cv2.bitwise_and(res2,green);
        
        crack = cv2.addWeighted(res3,float(0.5), Image,float(0.5),0)
        
        cv2.imwrite('./output\\4.crackResult\\' + str(gaus_file_list[k]), crack)
        
        k += 1
        

In [225]:
def topHat():
    # 사진 경로 확정
    path_dir = './input'
    file_list = os.listdir(path_dir)
    i = 0
    images = glob.glob('input\*.jpg')
    
    for fname in images:
        im = cv2.imread(fname, 0)
        egde = cv2.Canny(im,110,150)
        kernel = np.ones((5,5),np.uint8)
        tophat = cv2.morphologyEx(egde, cv2.MORPH_TOPHAT, kernel)
        cv2.imwrite('./output\\5.topHat\\' + str(file_list[i]), tophat)
        i += 1

In [236]:
def findLeak():
    topHat()
    i = glob.glob('input\*.jpg')
    k = 0
    for fanme in i:
        path_dir = './input'
        file_list = os.listdir(path_dir)
        Images = glob.glob('./input\\*.jpg')
        Image = cv2.imread(Images[k],1)
        
        green = cv2.imread('green.jpg', cv2.IMREAD_COLOR)
        
        gaus_path_dir = './output/1.gausThresh'
        gaus_file_list = os.listdir(path_dir)
        gausImages = glob.glob('./output\\1.gausThresh\\*.jpg')
        gausImage = cv2.imread(gausImages[k],1)
        
        topHat_path_dir = './output/5.topHat'
        topHat_file_list = os.listdir(path_dir)
        topHatImages = glob.glob('./output\\5.topHat\\*.jpg')
        topHatImage = cv2.imread(topHatImages[k],1)
        
        res = cv2.bitwise_and(gausImage, topHatImage)
        res2 = cv2.bitwise_and(res,green);

        leak = cv2.addWeighted(res2,float(0.5), Image,float(0.5),0)
        
        cv2.imwrite('./output\\6.leakResult\\' + str(gaus_file_list[k]), leak)
        
        k += 1
        

In [410]:
def cannyEdge():
    # 사진 경로 확정
    path_dir = './input'
    file_list = os.listdir(path_dir)
    i = 0
    images = glob.glob('input\*.jpg')
    
    for fname in images:
        im = cv2.imread(fname, 0)
        egde = cv2.Canny(im,0,150)
        cv2.imwrite('./output\\7.cannyEdge\\' + str(file_list[i]), egde)
        i += 1    

In [411]:
def findCoat():
    cannyEdge()
    i = glob.glob('input\*.jpg')
    k = 0
    for fanme in i:
        path_dir = './input'
        file_list = os.listdir(path_dir)
        Images = glob.glob('./input\\*.jpg')
        Image = cv2.imread(Images[k],1)
        
        green = cv2.imread('green.jpg', cv2.IMREAD_COLOR)
        
        gaus_path_dir = './output/1.gausThresh'
        gaus_file_list = os.listdir(path_dir)
        gausImages = glob.glob('./output\\1.gausThresh\\*.jpg')
        gausImage = cv2.imread(gausImages[k],1)
        
        canny_path_dir = './output/5.topHat'
        canny_file_list = os.listdir(path_dir)
        cannyImages = glob.glob('./output\\7.cannyEdge\\*.jpg')
        cannyImage = cv2.imread(cannyImages[k],1)
        
        res = cv2.bitwise_and(gausImage, cannyImage)
        res2 = cv2.bitwise_and(res,green);

        coat = cv2.addWeighted(res2,float(0.5), Image,float(0.5),0)
        
        cv2.imwrite('./output\\8.coatResult\\' + str(gaus_file_list[k]), coat)
        
        k += 1

In [420]:
def findCalculation():

SyntaxError: unexpected EOF while parsing (<ipython-input-420-953f73d2e8fb>, line 1)

In [412]:
def findALL():
    findCrack()
    findLeak()
    findCoat()

In [452]:
def printoutstuff(imagePath):
    img = cv2.imread(imagePath)
    
    # boundaries for the color red
    boundaries = [
        ([3, 75, 1], [60, 254, 65])
        ]
    
    for(lower, upper) in boundaries:
        # creates numpy array from boundaries
        lower = np.array(lower, dtype = "uint8")
        upper = np.array(upper, dtype = "uint8")
    
        # finds colors in boundaries a applies a mask
        mask = cv2.inRange(img, lower, upper)
        output = cv2.bitwise_and(img, img, mask = mask)
    
        # saves the image
        #cv2.imwrite('2'+img, output)
    
    tot_pixel = output.size
    green_pixel = np.count_nonzero(output)
    percentage = round(green_pixel * 100 / tot_pixel, 2)
    
    print("green pixels: " + str(green_pixel))
    print("Total pixels: " + str(tot_pixel))
    print("Percentage of green pixels: " + str(percentage) + "%")

k = 0
i = glob.glob('input\*.jpg')

for fanme in i:
    crack_path_dir = './output/4.crackResult'
    crack_file_list = os.listdir(path_dir)
    crackImages = glob.glob('.\output\\4.crackResult\\*.jpg')
    crackImage = cv2.imread(crackImages[k],1)
    print(crackImages[k])
    printoutstuff(crackImages[k])
    
    leak_path_dir = './output/6.leakResult'
    leak_file_list = os.listdir(path_dir)
    leakImages = glob.glob('./output\\6.leakResult\\*.jpg')
    leakImage = cv2.imread(crack_file_list[k],1)
    #printoutstuff(leakImages[k])
    
    coat_path_dir = './output/8.coatResult'
    coat_file_list = os.listdir(path_dir)
    coatImages = glob.glob('./output\\8.coatResult\\*.jpg')
    coatImage = cv2.imread(crack_file_list[k],1)
    #printoutstuff(coatImages[k])
    k += 1


    

.\output\4.crackResult\1.jpg
green pixels: 126532224
Total pixels: 126532224
Percentage of green pixels: 100.0%
.\output\4.crackResult\DSC01266.JPG
green pixels: 55632
Total pixels: 126532224
Percentage of green pixels: 0.04%
.\output\4.crackResult\DSC01267.JPG
green pixels: 16932
Total pixels: 126532224
Percentage of green pixels: 0.01%
.\output\4.crackResult\DSC01269.JPG
green pixels: 75
Total pixels: 126532224
Percentage of green pixels: 0.0%
.\output\4.crackResult\DSC01270.JPG
green pixels: 732
Total pixels: 126532224
Percentage of green pixels: 0.0%
.\output\4.crackResult\DSC01271.JPG
green pixels: 405255
Total pixels: 126532224
Percentage of green pixels: 0.32%
.\output\4.crackResult\DSC01272.JPG
green pixels: 13017
Total pixels: 126532224
Percentage of green pixels: 0.01%
.\output\4.crackResult\DSC01273.JPG
green pixels: 215373
Total pixels: 126532224
Percentage of green pixels: 0.17%
.\output\4.crackResult\DSC01275.JPG
green pixels: 10335
Total pixels: 126532224
Percentage of g

KeyboardInterrupt: 